In [25]:
from bs4 import BeautifulSoup as bs 
import urllib.request # urllib 호출 모듈
import requests
from urllib.parse import urlencode
from urllib.request import urlopen
import selenium, time, os
from selenium import webdriver
from urllib.request import urlretrieve
from tqdm import tqdm
import zipfile
from selenium.webdriver.common.keys import Keys
import sqlite3 
import json
import pandas as pd

# 엔터 신중히 누를것!!!!!

In [26]:
manufacturer = [] # 제조사
model = [] # 모델명
price = [] # 차량가격
carInfo = [] # 차량정보
year = [] # 연식
km = [] # 주행거리
fuel = [] #연료
transmission = [] # 변속기
efficiency = [] # 연비
type_car = [] # 차종
cc = [] # 배기량
color = [] # 색상
tax = [] # 세금미납
seize = [] # 압류
collateral = [] # 저당
res_num = [] # 제시번호
wreck = [] # 전손이력
water = [] # 침수이력
purpose = [] # 용도이력
owner = [] # 소유자변경
accident = [] # 보험사고정보 1 => 사고있음, 0 => 없음

# 엔터 신중히 누를것!!!!!

In [31]:
driver = webdriver.Chrome()
driver.get('https://www.kbchachacha.com/public/search/main.kbc#!?countryOrder=1&page=201&sort=-orderDate') # 국산차 페이지 접속
time.sleep(2)
driver.maximize_window()
time.sleep(1)

# order_page = driver.find_element_by_xpath('//*[@id="sort-orderDate"]') # 최신순 정렬
# order_page.click()

# time.sleep(1)
for num_fuckoff_error in range(21, 31): # 1부터 10페이지까지 한 세트
    # 페이지 이동 for문 시작
    for num_pages in range(1, 11):
        # 페이지 내부 차량 하나씩 클릭 for문 시작(for - 1)
        for num_cars in range(1, 26):
            carInfo_keys = []
            carInfo_vals = []
            first_tab = driver.window_handles[0]
            driver.switch_to.window(window_name=first_tab)
            find_car1 = driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div[3]/div[4]/div[2]/div[{num_cars}]/div[2]/div/a')
            find_car1.click()

            time.sleep(2)

            last_tab = driver.window_handles[-1]
            driver.switch_to.window(window_name=last_tab)
            targetSite = driver.current_url
            # print(targetSite) # 수집된 페이지 url표시
            # aa = urllib.request.urlopen(targetSite)
            # source = aa.read()
            source = driver.page_source # 페이지의 소스 가져오기
            source = bs(source, 'html.parser')

            source
            result1 = source.select('div.car-buy-price > strong')
            result2 = source.select('dd strong')
            result3 = source.select('div.car-buy-price div dl dd strong') # 중고차 차량 가격

            # 소스에서 가져오기

            carInfo_keys1 = source.select('div.detail-info01 > table > tbody > tr > th')
            carInfo_vals1 = source.select('div.detail-info01 > table > tbody > tr > td')
            accident_his = source.select('div.detail-info02 > div.mg-t40 > a.detail-txt-link02 > span.link-arrow') # 보험사고정보 따로 저장
            carInfo_keys2 = source.select('div.detail-info02 > div.mg-t40 > dl > dt')
            carInfo_vals2 = source.select('div.detail-info02 > div.mg-t40 > dl > dd')

            # 제조사, 모델명
            for i in result1:
                mm = i.text
                k = mm.split(')')
                ak = k[1].split(maxsplit = 1)
                carInfo_keys.append('제조사')
                manu = ak[0]
                carInfo_keys.append('모델명')
                model_name = ak[1]

            # 중고차 차량 가격
            for i in result3:
                tmp_price = i.text
            tmp_price

      
            
            # 왼쪽에 있는것들
            for i in carInfo_keys1:
                carInfo_keys.append(i.text)


            tem_car1 = []
            for i in carInfo_vals1:
                tem_car1.append(i.text)

            for i, v in enumerate(tem_car1):
                if i == 5:
                    v = v.split(maxsplit = 2)
                    carInfo_vals.append(v[0])
                elif i == 12:
                    v = v.split(maxsplit = 2)
                    carInfo_vals.append(v[0])
                else:
                    carInfo_vals.append(v)


            # 중간에 있는것들

            for i in carInfo_keys2:
                carInfo_keys.append(i.text)


            tem_car2 = []
            for i in carInfo_vals2:
                tem_car2.append(i.text)

            for i, v in enumerate(tem_car2):
                if i == 2:
                    v = v.split(maxsplit = 2)
                    carInfo_vals.append(v[0])
                else:
                    carInfo_vals.append(v)

            # 보험사고이력 따로 저장한 부분
            for i in accident_his:
                a = i.text
                b = '사고있음' in a
                if b :
                    accident_true = 1
                else :
                    accident_true = 0


            # 최종 리스트에 값 집어넣기
            if len(carInfo_vals) == 17:
                manufacturer.append(manu) # 따로 들어감
                model.append(model_name) # 따로 들어감
                price.append(tmp_price) # 따로 들어감
                carInfo.append(carInfo_vals[0])
                year.append(carInfo_vals[1])
                km.append(carInfo_vals[2])
                fuel.append(carInfo_vals[3])
                transmission.append(carInfo_vals[4])
                efficiency.append(carInfo_vals[5])
                type_car.append(carInfo_vals[6])
                cc.append(carInfo_vals[7])
                color.append(carInfo_vals[8])
                tax.append(carInfo_vals[9])
                seize.append(carInfo_vals[10])
                collateral.append(carInfo_vals[11])
                res_num.append(carInfo_vals[12])
                wreck.append(carInfo_vals[13])
                water.append(carInfo_vals[14])
                purpose.append(carInfo_vals[15])
                owner.append(carInfo_vals[16])
                accident.append(accident_true)



            print(f'{num_fuckoff_error * 10 + num_pages} 페이지 === {num_cars}번째 차량 정보 수집 완료') # 총 17개 컬럼
            driver.close() # 차량 정보 수집 후 탭 닫기
        if num_pages == 10:
            driver.switch_to.window(window_name=first_tab) # 첫 탭으로 이동 
            next_chapter = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div[2]/div[2]/div[3]/div[5]/a[11]')
            next_chapter.click()
            print(f'=========================================={num_fuckoff_error * 10 + num_pages} 페이지 이동 완료==================================================')
            time.sleep(1)

            # else:
            # 첫번째 for 문 끝 (for - 1)

            # driver.switch_to.window(window_name=first_tab) # 첫 탭으로 이동 
            # next_page = driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div[3]/div[5]/a[{num_pages+1}]') # 다음 페이지 이동
            # # next_page.click()
            # print(f'=========================================={num_pages+1} 페이지 이동 완료==================================================')
            # time.sleep(1)
        else :
            driver.switch_to.window(window_name=first_tab) # 첫 탭으로 이동 
            next_page = driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div[3]/div[5]/a[{num_pages+1}]') # 다음 페이지 이동
            next_page.click()
            print(f'=========================================={num_fuckoff_error * 10 + num_pages} 페이지 이동 완료==================================================')

211 페이지 === 1번째 차량 정보 수집 완료
211 페이지 === 2번째 차량 정보 수집 완료
211 페이지 === 3번째 차량 정보 수집 완료
211 페이지 === 4번째 차량 정보 수집 완료
211 페이지 === 5번째 차량 정보 수집 완료
211 페이지 === 6번째 차량 정보 수집 완료
211 페이지 === 7번째 차량 정보 수집 완료
211 페이지 === 8번째 차량 정보 수집 완료
211 페이지 === 9번째 차량 정보 수집 완료
211 페이지 === 10번째 차량 정보 수집 완료
211 페이지 === 11번째 차량 정보 수집 완료
211 페이지 === 12번째 차량 정보 수집 완료
211 페이지 === 13번째 차량 정보 수집 완료
211 페이지 === 14번째 차량 정보 수집 완료
211 페이지 === 15번째 차량 정보 수집 완료
211 페이지 === 16번째 차량 정보 수집 완료
211 페이지 === 17번째 차량 정보 수집 완료
211 페이지 === 18번째 차량 정보 수집 완료
211 페이지 === 19번째 차량 정보 수집 완료
211 페이지 === 20번째 차량 정보 수집 완료
211 페이지 === 21번째 차량 정보 수집 완료
211 페이지 === 22번째 차량 정보 수집 완료
211 페이지 === 23번째 차량 정보 수집 완료
211 페이지 === 24번째 차량 정보 수집 완료
211 페이지 === 25번째 차량 정보 수집 완료
==========================================211 페이지 이동 완료==================================================
212 페이지 === 1번째 차량 정보 수집 완료
212 페이지 === 2번째 차량 정보 수집 완료
212 페이지 === 3번째 차량 정보 수집 완료
212 페이지 === 4번째 차량 정보 수집 완료
212 페이지 === 5번째 차량 정보 수집 완료
212 페이지 === 6번째 차량 정보 수집 완료
212 페이지 ==

In [34]:
# 수집데이터 데이터프레임으로 만들기

df_carInfo_test1 = pd.DataFrame({'manufacturer' :manufacturer,
                             'model' :model,
                             'price' : price,
                             'carInfo' :carInfo,
                              'year' : year,
                              'km' : km,
                              'fuel' : fuel,
                              'transmission' : transmission,
                              'efficiency' : efficiency,
                              'type_car' : type_car,
                              'cc' : cc,
                              'color' : color,
                              'tax' : tax,
                              'seize' : seize,
                              'collarteral' : collateral,
                              'res_num' : res_num,
                              'wreck' : wreck,
                              'water' : water,
                              'purpose' : purpose,
                              'owner' : owner,
                              'accident' : accident})


print('============================================= 데이터 프레임 저장 완료 ===========================================================')

============================================= 데이터 프레임 저장 완료 ===========================================================


In [39]:
# SQLite3 DB로 저장하기

con = sqlite3.connect("carInfo_test_201_3001.db")

df_carInfo_test1.to_sql("df_carInfo_test_201_3001", con, index=False)

con.close()

In [8]:
# SQLite3 DB 불러오기

con = sqlite3.connect("carInfo_test.db") # carInfo_test0603_1.db

df_carInfo = pd.read_sql("SELECT * FROM df_carInfo_test", con)

con.close()

print('================================================ 데이터 베이스 생성 완료 =============================================================')
print('=========================================== 데이터 프레임 명 : df_carInfo_test =======================================================')

================================================ 데이터 베이스 생성 완료 =============================================================
=========================================== 데이터 프레임 명 : df_carInfo_test =======================================================


In [ ]:
sqlite3 source.db .dump tmptable | sqlite3 target.db

In [ ]:
carInfo_vals

['59주0666',
 '16년02월(16년형)',
 '89,633km',
 '가솔린',
 '오토',
 '8.8Km',
 '대형',
 '3,342cc',
 '검정',
 '\n없음\n',
 '\n없음\n',
 '\n1 건\n\t\t\t\t\t\t\t\t            \n\t\t\t\t\t\t\t\t        \n\t\t\t\t\t\t\t\t\t',
 '20201050088',
 '\n없음\n',
 '\n없음\n',
 '없음',
 '\n\t                            \t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t  \t\t\t\t\n\t\t\t\t\t\t    \t\t\t\t1회\n\t\t\t\t\t\t  \t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t']

In [25]:
carInfo_keys

['차량정보',
 '연식',
 '주행거리',
 '연료',
 '변속기',
 '연비',
 '차종',
 '배기량',
 '색상',
 '세금미납',
 '압류',
 '저당',
 '제시번호',
 '전손이력',
 '침수이력 ',
 '용도이력',
 '소유자변경',
 '차량정보',
 '연식',
 '주행거리',
 '연료',
 '변속기',
 '연비',
 '차종',
 '배기량',
 '색상',
 '세금미납',
 '압류',
 '저당',
 '제시번호',
 '전손이력',
 '침수이력 ',
 '용도이력',
 '소유자변경']